<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Другое-разбиение-для-обучения-и-валидации" data-toc-modified-id="Другое-разбиение-для-обучения-и-валидации-0.0.1"><span class="toc-item-num">0.0.1&nbsp;&nbsp;</span>Другое разбиение для обучения и валидации</a></span></li><li><span><a href="#Квантильная-регрессия" data-toc-modified-id="Квантильная-регрессия-0.0.2"><span class="toc-item-num">0.0.2&nbsp;&nbsp;</span>Квантильная регрессия</a></span></li><li><span><a href="#Другой-подход---предсказание-ошибки" data-toc-modified-id="Другой-подход---предсказание-ошибки-0.0.3"><span class="toc-item-num">0.0.3&nbsp;&nbsp;</span>Другой подход - предсказание ошибки</a></span></li></ul></li></ul></li></ul></div>

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
%matplotlib inline
import seaborn as sns

from scipy.stats import boxcox, probplot
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('calls_dataset_with_features.csv')

In [3]:
data.head()

,address,duration_minutes,feat_1,feat_2,geometry,poly_side,atm,bank,bus_station,bus_stop,cafe,device_charging_station,office,public_service,restaurant,shelter,station,traffic_signals,waste_basket
0,8811aa602dfffff,5.092667,0.0,25.0,"POLYGON ((37.75103240708841 55.80603592669851,...",poly3,1.0,0.0,0.0,8.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,8811aa45d9fffff,6.037037,0.0,63.0,"POLYGON ((37.40263832553934 55.71842073952227,...",poly0,5.0,3.0,0.0,5.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
2,881181a235fffff,4.722727,0.0,11.0,"POLYGON ((37.74271427608124 55.43691297888912,...",poly1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8811aa722bfffff,5.868982,0.0,36.0,"POLYGON ((37.42241082476207 55.85767681439805,...",poly2,2.0,2.0,0.0,12.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8811aa714dfffff,2.937719,0.0,19.0,"POLYGON ((37.45469517717559 55.7843306893976, ...",poly2,1.0,0.0,0.0,9.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0


### Другое разбиение для обучения и валидации

Фиксируем данное разбиение и будем его использовать в дальнейшем

In [4]:
X_train = data.drop(['address', 'geometry', 
                     'poly_side','duration_minutes'],axis=1)[data['poly_side'].isin(['poly1','poly2'])]

y_train = np.log1p(data['duration_minutes'][data['poly_side'].isin(['poly1','poly2'])])

In [5]:
X_valid = data.drop(['address', 'geometry', 
                     'poly_side','duration_minutes'],axis=1)[data['poly_side'].isin(['poly0','poly3'])]

y_valid = np.log1p(data['duration_minutes'][data['poly_side'].isin(['poly0','poly3'])])

Оценим качество случайного леса на кросс валидации

In [6]:
from sklearn.model_selection import cross_val_predict

y_pred = cross_val_predict(RandomForestRegressor(n_estimators=100, bootstrap=True),
    X_train, y_train, cv=5, n_jobs=-1)

print('Metric:', np.sqrt(mean_squared_error(np.exp(y_train), np.exp(y_pred))))

Metric: 1.9379533753970475


Мы применяли различные алгоритмы для обучения и мы можем выбрать тот алгоритм, который минимизирует ошибку. На практике нам часто не требуется бесконечно улучшать качество алгоритма, например можно зафиксировать модель случайного леса и далее сосредоточиться на интерпретации результата. 

В задаче регрессии алгоритм возвращает одно число, 
но на практике нам достаточно с какой-то степенью уверенности сказать, что предсказываемое значение меньше какого-то порога. Для этого нам нужно найти интервал предсказания - такой интервал, в который с некоторой уверенностью попадает значение целевой переменной.

### Квантильная регрессия

в scikit-learn можно найти реализацию квантильной регрессии в алгоритме `Gradient Boosted Regressor`: для обучения нужно задать квантиль какого порядка мы хотим получить 

In [7]:
alpha = 0.95
clf = GradientBoostingRegressor(loss='quantile', alpha=alpha,
                                n_estimators=100, max_depth=3,
                                learning_rate=.1, min_samples_leaf=9,
                                min_samples_split=9)

#обучаем 1-ю модель для предсказания верхней границы
clf.fit(X_train, y_train)


#обучаем 2-ю модель для предсказания нижней границы
clf.set_params(alpha=1.0 - alpha)
clf.fit(X_train, y_train)


#обучаем модель для предсказания целевой переменной
clf.set_params(loss='ls')
clf.fit(X_train, y_train)

#предсказание
y_lower = clf.predict(X_valid)
y_upper = clf.predict(X_valid)
y_pred = clf.predict(X_valid)

В итоге получаем, что с 90% вероятностью предсказанные значения будут лежать в этих интервалах:

In [8]:
results = pd.DataFrame()

results['y_lower'] = y_lower
results['pred'] = y_pred
results['y_upper'] = y_upper

In [9]:
results.head()

,y_lower,pred,y_upper
0,1.332314,1.750483,2.195313
1,1.606032,1.789236,2.007094
2,1.360539,1.809732,2.197030
3,1.178550,1.739170,2.095096
4,0.682342,1.694541,2.166694


Основной минус данного подхода - для разных квантилей нужно обучать отдельные модели. Поэтому ниже мы рассмотрим подход, который позволит сократить количество моделей и с помощью этого подхода можно находить интервал предсказания для разных квантилей.

### Другой подход - предсказание ошибки

Делаем две модели - первая модель предсказывает целевую переменную, а вторая модель предсказывает ошибку

In [10]:
X1, X2, y1, y2 = train_test_split(X_train, y_train, test_size=0.2)

In [11]:
clf_mean = RandomForestRegressor(criterion='mae')
clf_mean.fit(X1, y1)

y2_pred = clf_mean.predict(X2)
y2_pred_error = (y2 - y2_pred)**2

clf_std = RandomForestRegressor(criterion='mse')

clf_std.fit(X2, y2_pred_error)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

Находим интервалы на отложенной выборке

In [12]:
y_test_mean = clf_mean.predict(X_valid)
y_test_stdev = clf_std.predict(X_valid)

Далее находим интервал +/- 2 сигмы

In [13]:
results = pd.DataFrame()
results['predict_lower'] = y_test_mean - (2*(y_test_stdev)**0.5)
results['predict'] = y_test_mean
results['predict_upper'] = y_test_mean + (2*(y_test_stdev)**0.5)

In [14]:
results.head()

,predict_lower,predict,predict_upper
0,1.077062,1.701941,2.326820
1,1.457680,1.757029,2.056378
2,1.178265,1.743278,2.308292
3,0.761148,1.806959,2.852770
4,0.378254,1.805174,3.232093
